In [1]:
import pandas as pd

In [2]:
!ls

Untitled.ipynb
edit_success_20210303.04.03.1614746994_model_epoch0_ts10000.20210303.04.03.1614746138
edit_success_20210303.04.03.1614747484_model_epoch0_ts10000.20210303.04.03.1614746138
edit_success_20210303.05.03.1614747702_OTS


In [3]:
ots = pd.read_csv("edit_success_20210303.05.03.1614747702_OTS")

In [4]:
model = pd.read_csv("edit_success_20210303.04.03.1614747484_model_epoch0_ts10000.20210303.04.03.1614746138")

In [9]:
ots.success.mean()

0.7346938775510204

In [10]:
model.success.mean()

0.6938775510204082

In [17]:
ots.apply(lambda x: (x.new_ppl - x.orig_ppl)/x.orig_ppl, axis=1).mean()

0.018858077941799976

In [18]:
model.apply(lambda x: (x.new_ppl - x.orig_ppl)/x.orig_ppl, axis=1).mean()

-0.32517036485921913

In [21]:
ots.new_ppl.mean()

64.65336686737683

In [22]:
model.new_ppl.mean()

235.74996605697945